In [ ]:
import scanpy as sc
import pandas as pd
from scipy.sparse import issparse
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
pip install git+https://github.com/theislab/cpa

In [ ]:
import sys
import sklearn
import numpy as np
import os
import cpa
import matplotlib.pyplot as plt


In [ ]:
sc.settings.set_figure_params(dpi=100)

In [ ]:
adata = sc.read("/work/CPA_Healthy_hamstring/new_data_raw_fix/scAtlas_Human_vascular_cells_processed_RAW_1.h5ad")

In [ ]:
adata

In [ ]:
# Split the data set into train and test
from sklearn.model_selection import train_test_split

split_key = "split"
adata.obs[split_key] = "train"
idx = list(range(len(adata)))
idx_train, idx_test = train_test_split(adata.obs_names, test_size=0.1, random_state=42)
adata.obs.loc[idx_train, split_key] = "train"
adata.obs.loc[idx_test, split_key] = "test"

In [ ]:
adata_train = adata[adata.obs[split_key] == "train"].copy()
adata_test = adata[adata.obs[split_key] == "test"].copy()

In [ ]:
adata.obs["donor_id"]

In [ ]:
adata.obs['split'].value_counts()

In [ ]:
adata.layers['counts'] = adata.X.copy()

In [ ]:
adata.X = adata.layers["counts"].copy()

In [ ]:
import scanpy as sc

# Assuming 'adata' is your AnnData object
# Extract the CPA_pred from the obsm attribute
adata_CPA_pred = adata.copy()  # Make a copy to avoid modifying the original object
adata_CPA_pred.obsm['CPA_pred'] = adata.obsm['CPA_pred']  # Ensure the CPA_pred is included

# Save the new AnnData object to a new .h5ad file
adata_CPA_pred.write('adata_with_CPA_pred_scAtlas.h5ad')


In [ ]:
cpa.CPA.setup_anndata(adata,
                      perturbation_key='donor_id',
                      control_group='Hs001',
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type', 'self_reported_ethnicity','bmi_group',
                                                  'donor_id', 'surgery', 'fat_type', 'tissue'],
                      max_comb_len=1,
                     )

In [ ]:
model_params = {
    "n_latent": 64,
    "recon_loss": "nb",
    "doser_type": "linear",
    "n_hidden_encoder": 128,
    "n_layers_encoder": 2,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 2,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": False,
    "use_layer_norm_decoder": True,
    "dropout_rate_encoder": 0.0,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 42,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 0,
    "mixup_alpha": 0.0,
    "adv_steps": None,
    "n_hidden_adv": 64,
    "n_layers_adv": 3,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 5.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": True,
    "gradient_clip_value": 1.0,
    "step_size_lr": 10,
}

In [ ]:
adata.obs['split'].value_counts()

In [ ]:
model = cpa.CPA(adata=adata,
                split_key='split',
                train_split='train',
                test_split='test',
                **model_params,
               )

In [ ]:
model.train(max_epochs=500,
            use_gpu=True,
            batch_size=512,
            plan_kwargs=trainer_params,
            early_stopping_patience=5,
            check_val_every_n_epoch=5,
            save_path='CPA_project/scAtlas_model_batch_removed',
           )

In [ ]:
model = cpa.CPA.load("/work/scAtlas_runs/CPA_ScAtlasdata/CPA_project/Results/", adata = adata)

In [ ]:
cpa.pl.plot_history(model)

In [ ]:
df = pd.read_csv("/work/scAtlas_runs/CPA_ScAtlasdata/CPA_project/Results/history.csv")

In [ ]:
sns.lineplot(df,x="epoch", y= "r2_mean", hue="mode")

In [ ]:
model = model.load("/work/scAtlas_runs/CPA_ScAtlasdata/CPA_project/Results_scAtlas_more_cov_new_Atlas_data/", adata = adata)

In [ ]:
cpa.CPA.setup_anndata(adata_test,
                      perturbation_key='donor_id',
                      control_group='Hs001',
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type', 'self_reported_ethnicity','bmi_group',
                                                  'donor_id', 'surgery', 'fat_type', 'tissue'],
                      max_comb_len=1,
                     )

In [ ]:
# Predict using cell_type embeddings, removing the batch embeddings (batch corrected)
output_no_batch = model.custom_predict(adata= adata_test,
                   covars_to_add=['cell_type', 'self_reported_ethnicity','bmi_group',
                                                  'donor_id', 'surgery', 'fat_type', 'tissue'],
                   add_batch=False,
                   add_pert=False,
                   batch_size=2048)


In [ ]:
CPA_pred = output_no_batch["latent_x_pred"].X

In [ ]:
CPA_pred

In [ ]:
import scanpy as sc

# Get the prediction matrix
CPA_pred = output_no_batch["latent_x_pred"].X
CPA_pred = CPA_pred.toarray() if hasattr(CPA_pred, "toarray") else CPA_pred

# Add prediction to adata_test
adata_test.obsm["CPA_pred"] = CPA_pred

# Save to file
adata_test.write("adata_ATLAS_with_CPA_pred_removed_batch.h5ad")
